In [1]:
# Colab
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir(f'/content/drive/MyDrive/proj-nlp-2024/data')

# # Local
# os.chdir('c:\\Users\\green\\Documents\\GitHub\\proj-nlp-2024\\data')

Mounted at /content/drive


In [2]:
import os
import json
from pandas.core.common import flatten
# from sentence_transformers import SentenceTransformer, CrossEncoder
import torch
# import hnswlib
import pickle
import numpy as np
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import accelerate
import pandas as pd

# Dataset

In [4]:
# Loading the json file
train_path = os.path.join(os.getcwd(), 'beerqa_train_v1.0.json')
with open(train_path, 'r') as file:
    train_dataset = json.load(file)

Exploring the dataset

In [5]:
train_dataset.get('data', {})[22]

{'id': 'cf01284af96aa29b6b8f098273b8f13122447ac0',
 'src': 'hotpotqa',
 'answers': ['the United Kingdom'],
 'question': 'Where is the group founded by Noel Stanton based?',
 'context': [['Jesus Army',
   'The Jesus Army, also known as the Jesus Fellowship Church and the Bugbrooke Community, was a neocharismatic evangelical Christian movement based in the United Kingdom, part of the British New Church Movement. The name "Jesus Army" was specifically used for the outreach and street-based evangelism for which they were known.'],
  ['Noel Stanton',
   'Stanton was born in Bedfordshire in the East of England, and educated at Bedford Modern School. His parents were farmers. When he was 18, he was conscripted into British military service with the Royal Navy. The navy sent him to Sydney, Australia, where he was approached by evangelist Frank Jenner, who asked him, "If you should die tonight, where would you go? Would it be heaven or hell?" Stanton felt convicted for several months afterwards

Turning it into a dataframe

In [7]:
dataset = pd.DataFrame(train_dataset['data'])
dataset.head()

,id,src,answers,question,context
0,8af07575b8444ae748634478f96b00d4e7dbd170,squad,[1793],When did Wordsworth initially attack Burke?,"[[Edmund Burke, In the 19th century, Burke was..."
1,7e07be1a905b70f143b2c8aff00caf16e2b32b31,squad,[Investiture Controversy],The clash between Henry IV and the pope was pa...,"[[Middle Ages, During the early High Middle Ag..."
2,c9dfc01655bc5abc59a1ee2583c8b3f7fba6c2e2,squad,[the Thermidorian Reaction],What historical event brought about the fall o...,"[[Napoleon, Some contemporaries alleged that B..."
3,cf10ffbb5023cd0bc4637655458512b2186dc886,squad,[Morales],Who came up with a policy for indigenous auton...,"[[Indigenous peoples of the Americas, Morales ..."
4,0a185aca81f9e36bfb2486f690885011a6aefb2e,hotpotqa,[Darren Lynn Bousman],"Greg Hoffman was working on ""Saw III,"" a 2006 ...","[[Saw III, Saw III is a 2006 horror film direc..."


Dropping useless columns

In [8]:
dataset = dataset.drop(columns=['id', 'src'])
dataset.head()

,answers,question,context
0,[1793],When did Wordsworth initially attack Burke?,"[[Edmund Burke, In the 19th century, Burke was..."
1,[Investiture Controversy],The clash between Henry IV and the pope was pa...,"[[Middle Ages, During the early High Middle Ag..."
2,[the Thermidorian Reaction],What historical event brought about the fall o...,"[[Napoleon, Some contemporaries alleged that B..."
3,[Morales],Who came up with a policy for indigenous auton...,"[[Indigenous peoples of the Americas, Morales ..."
4,[Darren Lynn Bousman],"Greg Hoffman was working on ""Saw III,"" a 2006 ...","[[Saw III, Saw III is a 2006 horror film direc..."


Creation of boolean column for linear classification based on (?)

In [15]:
dataset['context'][22][2]

['Noel Stanton',
 'When World War II ended, Stanton attended All Nations Bible College and worked for and then went into business. In 1957, he became the pastor of a Baptist church in Bugbrooke, Northamptonshire. Under Stanton\'s leadership, the church took on characteristics of the Charismatic Movement and then of the 1960s counterculture. In 1973, he began turning the church into an intentional community modelled after early Christianity, and the resulting movement became the Jesus Army. He wrote the book "Your Baptism Into Jesus Christ and His Church", which was published in 1998. Stanton remained the Jesus Army\'s leader until 2009, when he named Mick Haines the new leader before dying on 20 May.']

# Linear classifier trained on data

# Transformer-based classifier